### 動態公車調度模型簡介

##### 研究問題
>問題：目前固定班次的公車無法應付乘客突然暴增的情形，故需要藉由電子票證的歷史資料，以及影像辨識的即時資料，搭配預測模式與時空路網模型，提前告知depot發加班車的合適時間點。

>描述：針對高雄市公車路網，利用過去電子票證資料預測站間量，與部份車站設置監視器的影像辨識系統，所提供即時的候車人數預測上下車人數。由於高雄市公車去回之候車空間是分開的，在此當不同站點做區隔。

>輸入：
	預測時間t至t+Δt_2之每兩站的站間量(由預測得到的上下車人數推得)
	在有設監視器的車站預測時間t至t+Δt_2該站的乘客抵達率，未設監視器的車站則透過過去票證資料補齊。
	預測時間段期間，站點間的旅行時間
    
>旅客行為：等太久會隨時間依比例流失，Δt_2假定一小時.

>業者行為：派車、有賺錢才發車(加班車不會被補助，原班次仍有政府補助)。

>目標式：最大化利潤(票證收入-發車成本)。

>輸出：指令形式(每一個時階發車與否)，t至t+Δt_2 
	目前有兩台影像辨識攝影機與三條公車路線(如下圖示意圖)
 
##### 研究假設
	暫不考慮車隊數
	假設加班車容量遠大於加班車之需求(故加班車不會有容量上限，僅原班次有容量上限，由可接的擁擠量-站間量而得)
    
##### 解決方案
建立時空路網模型，以解決派遣公車加班車的問題。

>原則
1.	透過模式的目標式，最大化業者派遣加班車的利潤(票證收入-發車成本)。
2.	透過兩大時空路網(公車加班車時空路網與乘客時空路網)確保公車加班車調度的合理性。
##### 時空路網概念
>兩大路網
1.	公車加班車的路網：由於高雄市公車去回之候車空間是分開的，在此當不同站點做區隔。
2.	乘客的路網：由於無法推得乘客的迄點，故在乘客路網中所有乘客的迄點均為最後一站，並假設加班車容量遠大於加班車之需求，故不考慮加班車的容量限制。
在兩大時空路網中，路網圖的X軸代表站點的次序，Y軸代表預測時間段的次序。此模式中的公車加班車路網表示加班車的派遣與流動，其中的旅行arcs由事先預測得到的旅行時間參數建立(如2.3的圖所示)；乘客路網則表示對加班車有需求的乘客流動，而其中的旅行arcs同公車加班車路網中的旅行arcs，乘客路網中的demand 流失 arcs，則與其路網中旅客等候arcs同時並存，也就是當某一站點時間段有一條等候arc往下連到下個時間段，該node就會同時有一條demand 流失 arc連到sink node，以利表示乘客會隨等候時間的產生，而等比例流失(如2.4的圖所示)。

##### 模式方向
	本計劃案主要著重在於預測原固定班次的站間量(用預測得到的上下車人數推)與預測每站的乘客抵達率的合理性(參數)，利用可事先預測的參數建立合適的模型。
	由於加班車不會受到政府補助，此時空路網模式是站在業者的角度去做派遣公車加班車的決策


# 預測資料處理

>載入讀取資料的套件

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

>讀取預測資料

In [2]:
busfactor = pd.read_csv('testdata.csv',encoding='BIG5')
busfactor.drop('Unnamed: 0', axis = 1, inplace = True)

In [3]:
busfactor.head()

,日期,班次,StopName_Zh_tw,StopSequence,UP,DOWN,PASSNUM,pedict_up,pedict_down,pedict_passnum,drive_passnum
0,2019/7/1,900,前鎮站,1,1,0,1,3,0,3,3
1,2019/7/1,900,輕軌夢時代站(統一時代百貨),2,0,0,1,0,0,2,3
2,2019/7/1,900,台糖物流,3,0,0,1,0,0,2,3
3,2019/7/1,900,輕軌經貿園區站,4,0,0,1,0,0,2,3
4,2019/7/1,900,正勤路,5,1,0,2,2,0,4,5


>將預測資料轉成模式參數

In [4]:
#旅行時間
timeable={'前鎮站':'800','輕軌夢時代站(統一時代百貨)':'801','台糖物流':'802','輕軌經貿園區站':'803','正勤路':'803'
          ,'正勤社區':'804','中華復興路口':'805','中華五路':'806','新光路口(圖書總館)':'807','三多四路口':'808'
         ,'大遠百百貨(捷運三多商圈站)':'810','新光三越(捷運三多商圈站)':'811','三多三路':'812','仁愛三街口':'813'
         ,'三多復興路口':'814','三多市場(三多三路)':'815','光華路口(光華夜市)':'816','林德官(三和市場)':'816'
         ,'和平路口':'817','五權國小(三多二路)':'818','凱旋二路口':'818','國際商工':'819','三信家商':'820'
         ,'五塊厝(三多一路)':'821','輔仁路口(三多一路)':'822','中正體育場':'823','衛武營':'824','衛武營國家藝術文化中心':'825'
         ,'衛武社區':'828','鳳山行政中心(澄清路)':'830','覺民路口(澄清路)':'832','褒揚街口':'833','澄清路':'835'
         ,'大華村':'836','正修科大(澄清湖)':'838','長庚復健大樓':'844','長庚醫院':'846','澄清湖棒球場':'848'}
time=timeable.values()

###### 整合投入資料

In [5]:
#原班次的班距(分鐘)
schedule=15
#原班次的起始時間
st=[]
for i in busfactor['班次']:
    st.append(i)
st=list(set(st))
clean=min(st)
for i,j in enumerate(st):
    st[i]-=clean
day= '2019/7/1'

>1. 投入：預測上車人數、下車人數、站間量

In [6]:
#7/1資料 busfactor.iloc[15]
m1=dict()
for i,j in enumerate(busfactor['日期']):
    if j == day:
        stop = busfactor.iloc[i]['StopName_Zh_tw']
        m1[stop]=dict()
        m1[stop]['pedict_up']=dict()
        m1[stop]['pedict_down']=dict()
        m1[stop]['drive_passnum']=dict()
for i,j in enumerate(busfactor['日期']):
    if j == day:
        stop = busfactor.iloc[i]['StopName_Zh_tw']
        for sch in st:
            if busfactor.iloc[i]['班次'] == sch+clean:
                m1[stop]['pedict_up'][sch]=busfactor['pedict_up'][i]
                m1[stop]['pedict_down'][sch] =busfactor['pedict_down'][i]
                if busfactor['drive_passnum'][i] < 0:
                    m1[stop]['drive_passnum'][sch] =busfactor['pedict_passnum'][i]
                else:
                    m1[stop]['drive_passnum'][sch] =busfactor['drive_passnum'][i]

In [7]:
stop=[]
for i in m1:
    stop.append(i)

>2.投入:旅行時間

In [8]:
#預測45分鐘，因為許多站間的旅行時間為一分鐘，甚至不到，因此在路網時間軸以1分鐘為一單位。
T=45
delta=1
#set K M B VA VN MTA
#觀測站點(K)
K=stop
#建立兩大路網中的node
demand=[]
for m in m1:
    for l in range(0,T):
        demand.append((m,l))

In [9]:
for i,j in enumerate(stop):
    if i == 0:
        m1[j]['travel_time']= 0
    else:
        m1[j]['travel_time']= int(timeable[j])-int(timeable[stop[i-1]])

>3.投入：抵達率

In [10]:
#抵達率(人/分鐘)
for i in m1:
    m1[i]['arrival_rate']=dict()
    for j in st:
        for k in range(j,j+schedule):
            m1[i]['arrival_rate'][k]=m1[i]['pedict_up'][j]/schedule
#四捨五入round()

##### 建立兩大路網

>乘客路網

In [11]:
#![title](bus_network.jpg)
#<img src="bus_network.jpg" width="100%">

<img src="passenger_network.png" width="80%">

1.等候時間的上限：透過不同起點建立一層路網做限制[迄點均假設是最後一站]

2.收入計算，針對加班車(非原班次的公車)，其所有路段的[流出-流入]*票證收入(16元)


In [12]:
#建立乘客路網的arc(B)
B=[]
for i in demand:
    tt = i[1]
    a=K.index(i[0])
    b=K.index(K[-1])
    t=i[1]
    for l in range(a,b):
        if a != b :
            t+= m1[K[l+1]]['travel_time']
            if t <= T:
                B.append(((K[l],tt),(K[l+1],t)))
            elif tt <= T:
                B.append(((K[l],tt),'dummy'))
            tt = t
for m in demand:
    B.append(('source',m))
    B.append((m,'sink'))
for t in range(0,T):
        B.append((('%s'%(stop[-1]),t),'dummy'))
for i in K:
    for a in range(T-1):
        B.append(((i,a),(i,a+1)))
B.append(('sink','dummy'))

>載入加班車模式套件

In [13]:
from gurobipy import *
import math

>公車路網

<img src="bus_network.jpg" width="80%">

In [14]:
#建立公車路網的所有arc(VV)&公車屬於加班車的載客arc(VAB)&固定班次的載客arc(SVS)
VV=tuplelist()
VAB=tuplelist()
SVS=tuplelist()
k=stop[0]
for i in range(0,T):
    VV.append(('source',(k,i)))     
VV.append(('source','sink'))
for i in range(0,T):
    for m,l in enumerate(K):
        if i != T and K[m] != K[-1]:
            if i+m1[K[m+1]]['travel_time'] <= T:
                VV.append(((l,i),(K[m+1],i+m1[K[m+1]]['travel_time'])))
                #檢查是否為原班次
                for check in st:
                    for cc in range(K.index(l)+1):
                        check+= m1[K[cc]]['travel_time']
                    if i != check:
                        VAB.append(((l,i),(K[m+1],i+m1[K[m+1]]['travel_time'])))
                    else :
                        SVS.append(((l,i),(K[m+1],i+m1[K[m+1]]['travel_time'])))
            else:
                VV.append(((l,i),'sink'))
for k in K:
    VV.append(((k,T),'sink')) 
#VS & VD
VS=['source']
VD=['sink']            
#VN       
VN=[]
for k in K:
    for t in range(0,T):
        VN.append((k,t))
#MN
MN=[]
for i in VAB:
    MN.append(i[0])
    MN.append(i[1])
MN=list(set(MN))

>參數

In [15]:
#param
rev=-16
km= 5
cc=38*km        
e=dict()
for k,j in enumerate(demand):
    e[j] = m1[j[0]]['arrival_rate'][j[1]]
        
q=10
veh=3
aa=6.0
MM=100

>站間量

## 模型

>假設
1.	已知：原固定班次站間的乘客量、乘客到站點的抵達率、各時間段的公車旅行時間
2.	透過事先定義好的公車可接受載量、預測得到的資料[站間量、上下車人數]，推算原班次在站間的容量，暫不考慮車隊數。
3.	假設加班車容量遠大於加班車之需求，故針對加班車無容量限制。


<img src="indiceandsets.JPG" width="80%">

<img src="paramandvarandobj.JPG" width="80%">

In [16]:
#variable
m = Model("realcontrol_6.0")
#m.setParam('MIPGAP',0.5)
x = {}
for i in VV:
    x[i[0],i[1]] = m.addVar(vtype = GRB.INTEGER,lb=0, name = "x[%s,%s]" %(i[0],i[1]))
y = {}
for i in B:
    y[i[0],i[1]] = m.addVar(vtype = GRB.CONTINUOUS,lb=0, name = "y[%s,%s]" %(i[0],i[1]))

Using license file C:\Users\James\gurobi.lic
Academic license - for non-commercial use only


<img src="st1.JPG" width="80%">

<img src="st2.jpg" width="80%">

(1)	在公車路網中，從source node 出去的車子數為veh

(2)	在公車路網中，回到sink node的車子數為veh

(3)	公車路網的流量平衡

(4)	乘客路網的流量平衡

(5)	乘客路網的每個時空站點抵達乘客數

(6)	原班次公車的載客量不能超過車子的容量[這裡的容量為可接受壅擠量扣除站間量所剩餘的載客量]

(7)	有加班車乘客才能移動

(8)	乘客會因為等候而等比例流失

In [17]:
#obj
m.setObjective(sum((sum(y[a,h] for a,h in B if a ==j  and h != 'sink' and h != 'dummy' and a[0] != h[0] )\
            -sum(y[i,b] for i,b in B if b == j and i[0] != b[0]\
            and i != 'source'  and b != 'sink'and b[1] != T)) \
            for j in MN )*rev +\
            sum(cc*x[i[0],i[1]] for i in VV if i[0] == 'source' and i[1] != 'sink')  , GRB.MINIMIZE)

#subjectto
for i in VS:
    m.addConstr(quicksum(x[i,j] for i,j in VV.select(i,'*')) == veh)
    
for j in VD:
    m.addConstr(quicksum(x[i,j] for i,j in VV.select('*',j)) == veh)

for i in VN:
    m.addConstr(quicksum(x[h,j] for h,j in VV if h == i) - quicksum(x[j,l] for j,l in VV if l == i) == 0)
    

for i in MN:
    if i[1] <= T:
        m.addConstr(quicksum(y[h,j] for h,j in B if h==i) - quicksum(y[j,l] for j,l in B if l == i) == 0)

for a,k in enumerate(demand):
    m.addConstr(y['source',k] == e[k]*(k[1]%15))

for i in SVS:
    if i[0][1] <= T:
        if (q-m1[i[0][0]]['drive_passnum'][i[0][1]-i[0][1]%schedule]) >= 0:
            m.addConstr(y[i[0],i[1]]  <= (q-m1[i[0][0]]['drive_passnum'][i[0][1]-i[0][1]%schedule]))
        else:
            m.addConstr(y[i[0],i[1]]  <= 0)
for i in VAB:
    if i[0][1] <= T:
        m.addConstr(y[i[0],i[1]]  <= MM*x[i[0],i[1]])           
for i in B:
    if i[0][0] == i[1][0]:
        m.addConstr(aa*y[i[0],'sink'] == y[i[0],i[1]])

>求解

In [18]:
m.optimize()
for v in m.getVars():
    if v.x >= 0.01:
        print ('%s:%f' %(v.varName,v.x))
print ('objective value:%f'  %(m.objVal))

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 11789 rows, 29697 columns and 28456 nonzeros
Model fingerprint: 0x2076d549
Variable types: 27948 continuous, 1749 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+01, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-02, 8e+00]
Presolve removed 9952 rows and 27845 columns
Presolve time: 0.03s
Presolved: 1837 rows, 1852 columns, 5470 nonzeros
Variable types: 1807 continuous, 45 integer (2 binary)

Root relaxation: objective -9.217336e+03, 2819 iterations, 0.06 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -9217.3364    0   30          - -9217.3364      -     -    0s
H    0     0                    -3545.789482 -9158.2856   158%     -    0s
     0     0 -8629.5739    0   33 -3545.7895 -8629.5739   143%     -    0s
H    0   

y[('中正體育場', 43),sink]:0.466667
y[source,('中正體育場', 44)]:0.933333
y[('中正體育場', 44),sink]:3.733333
y[source,('衛武營國家藝術文化中心', 1)]:0.133333
y[('衛武營國家藝術文化中心', 1),sink]:0.019048
y[source,('衛武營國家藝術文化中心', 2)]:0.266667
y[('衛武營國家藝術文化中心', 2),sink]:0.054422
y[source,('衛武營國家藝術文化中心', 3)]:0.400000
y[('衛武營國家藝術文化中心', 3),sink]:0.103790
y[source,('衛武營國家藝術文化中心', 4)]:0.533333
y[('衛武營國家藝術文化中心', 4),sink]:0.165153
y[source,('衛武營國家藝術文化中心', 5)]:0.666667
y[('衛武營國家藝術文化中心', 5),sink]:0.236798
y[source,('衛武營國家藝術文化中心', 6)]:0.800000
y[('衛武營國家藝術文化中心', 6),sink]:0.317256
y[source,('衛武營國家藝術文化中心', 7)]:0.933333
y[('衛武營國家藝術文化中心', 7),sink]:0.405267
y[source,('衛武營國家藝術文化中心', 8)]:1.066667
y[('衛武營國家藝術文化中心', 8),sink]:0.499752
y[source,('衛武營國家藝術文化中心', 9)]:1.200000
y[('衛武營國家藝術文化中心', 9),sink]:0.599788
y[source,('衛武營國家藝術文化中心', 10)]:1.333333
y[('衛武營國家藝術文化中心', 10),sink]:0.704580
y[source,('衛武營國家藝術文化中心', 11)]:1.466667
y[('衛武營國家藝術文化中心', 11),sink]:0.813450
y[source,('衛武營國家藝術文化中心', 12)]:1.600000
y[('衛武營國家藝術文化中心', 12),sink]:0.925814
y[source,('

### Question

>1.乘客四捨五入的機制

>2.旅行時間小於一分鐘的站，會有問題嗎?

>3.確認個別守衡，是每個起點都一個路網[路網中的等候arc會隨業者等候時長的政策而改變，因此不需靠限制式限制等候時間上限]?[迄點固定在最後一站]